Given a pickle file with parameters saved by the function fit_init_models, this will load those parameters and rerun the analysis for those exact parameters.  It will then save a list of all events that went into the analysis as well as a collection of files with calculated dff for the full volume at each time point.

# Imports

In [1]:
import pathlib
import pickle

import imageio
import numpy as np
import os

from janelia_core.dataprocessing.dataset import ROIDataset
from keller_zlatic_vnc.whole_brain.spontaneous import fit_init_models

## Parameters go here

In [2]:
# Provide the parameters that uniquely identifying the analysis we want to rerun

# Directory of the collection of results
results_dir = r'C:\Users\SiMView\Desktop'

# Folder with the saved parameters for the analysis we want to run again.
parameters_folder = r'parameters_map_type_orig_fit_window_length_1_window_offset_-1_pool_succeeding_turns_False_pool_preceeding_turns_False_co_th_3_q_end_offset_1'

# Update data read and write locations
annot_folders = [r'U:\bishoplab\projects\keller_drive\keller_vnc\data\full_annotations\em_volume_behavior_csv']
volume_loc_file = r'U:\bishoplab\projects\keller_drive\keller_vnc\data\EM_volume_experiment_data_locations.xlsx'
save_folder = r'U:\bishoplab\projects\keller_drive\data_transfer\CW_18-02-15_events_and_dff'

# Location of the original dataset holding the processed data
orig_dataset = r'W:\SV4\CW_18-02-15\L1-561nm-openLoop_20180215_163233.corrected\extracted\dataset.pkl'

# Name of ROI group in the original dataset that was analyzed
roi_group = 'brain_rois_1_5_5'

# Subject we want to save dff for
dff_subject = 'CW_18-02-15-L1'

## Now we find the parameters for that analsysis

In [3]:
with open(pathlib.Path(results_dir) / parameters_folder / 'mdl_fitting_params.pkl', 'rb') as f:
    params = pickle.load(f)
    
# Overwrite the parameters for data read and write locations
params['annot_folders'] = annot_folders
params['volume_loc_file'] = volume_loc_file
params['save_folder'] = save_folder
params['save_name'] = 'verification.pkl'

## Now we rerun the analysis

In [4]:
rs, analyze_annots, dff = fit_init_models(params, return_dff=True)

Gathering neural data for subject CW_18-02-15-L1
Performing stats for only one subject.


## Save annotation output

In [5]:
# Save csv file of annotation information (stripping dff values)

annotation_files = pathlib.Path(save_folder) / 'included_event_annotations.csv'

save_annots = analyze_annots.drop(columns='dff')
with open(annotation_files, 'wb') as f:
    save_annots.to_csv(f)

## Save tiff files of dff

In [6]:
with open(orig_dataset, 'rb') as f:
    dataset = ROIDataset.from_dict(pickle.load(f))

## Save collection of dff volumes

In [7]:
dff_folder = pathlib.Path(save_folder) / 'dff'
os.mkdir(dff_folder)

In [8]:
rois = dataset.roi_groups[roi_group]['rois']
n_rois = len(rois)

mn_img = dataset.stats['mean']
im_shape = mn_img.shape
subject_dff = dff[dff_subject]
n_time_pts = subject_dff.shape[0]
for t in range(n_rois):
    dff_image = np.zeros(im_shape, dtype=np.float32)
    for r_i in range(n_rois):
        cur_voxel_inds = rois[r_i].voxel_inds
        dff_image[cur_voxel_inds] = subject_dff[t, r_i]
        
    dff_image_file = dff_folder / 't_{:d}.tiff'.format(t)
    imageio.mimwrite(dff_image_file, dff_image)
    

IndexError: index 10367 is out of bounds for axis 0 with size 10367